In [1]:
import pandas as pd
import numpy as np
import os
import itertools
from ipynb.fs.defs.MMRWPC_v2 import MMRWPC
from ipynb.fs.defs.CrossAdaptiveGA import *
from sklearn.feature_selection import mutual_info_classif as mi_clasf
from scipy.stats import ttest_ind
from scipy.stats import ttest_rel
import pickle
import time
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

/home/a20114261/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/a20114261/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [ ]:
def process_MMRWPC(X):
#     global x_train
#     global x_test
    t_start = time.time()
    MMPC_RW_model = MMRWPC(0.05,0.0,1)
    MMPC_RW_model.fit(X)
    result = MMPC_RW_model.evaluate(0)
    nodes = {}
    nodes['Node1@2']={
        'nodes':[x['name'] for x in itertools.chain.from_iterable(result['CPC'])],
        'time':result['time']
                   }
    t_stop = time.time()
    
    return {
        'nodes':nodes,
        'time':t_start-t_stop,
    }

In [ ]:
def process_MI(X):
    t_start = time.time()
    mi_score = mi_clasf(X, X[list(X.keys())[0]],discrete_features=True)
    mi_results = list(zip(list(map(lambda x:'Node'+str(x+1),np.arange(1,541).tolist())),mi_score[1:]))
    t_stop = time.time()
    return {
        'nodes':mi_results,
        'time':t_stop-t_start,
    }

In [ ]:
def process_TTEST_ind(X):
    t_start = time.time()
    ttest_results = []
    x_keys = list(X.keys())
    x_pvalue_key = x_keys[0]
    x_keys.pop(0)
    for i in range(0,len(x_keys)):
        ttest_results.append(['Node'+str(i+2),ttest_ind(X[x_pvalue_key],X[x_keys[i]])[1]])
    t_stop = time.time()
    return {
        'nodes':ttest_results,
        'time':t_stop-t_start,
    }

In [ ]:
def process_TTEST_rel(X):
    t_start = time.time()
    ttest_results = []
    x_keys = list(X.keys())
    x_pvalue_key = x_keys[0]
    x_keys.pop(0)
    for i in range(0,len(x_keys)):
        ttest_results.append(['Node'+str(i+2),ttest_rel(X[x_pvalue_key],X[x_keys[i]])[1]])
    t_stop = time.time()
    return {
        'nodes':ttest_results,
        'time':t_stop-t_start,
    }

In [ ]:
def process_batch(batch_str):
    X = pd.read_csv(batch_str,delimiter=',', engine='python')
    X['Pvalue'] = list(map(lambda x: 1 if x<=0.01 else 0,list(X['Pvalue'])))
    del X['CauseGene']
    del X['EffectGene']
    del X['Replicate']
    del X['Treatment']
    
    
    results = {}
    results['MI'] = process_MI(X)
    results['TTEST_ind'] = process_TTEST_ind(X)
    results['TTEST_rel'] = process_TTEST_rel(X)
    results['MMRWPC'] = process_MMRWPC(X)
    return results
    

In [3]:
def making_x_train(directory):
    batch_arr = []
    for batch_str in os.listdir(directory):
        if (len(batch_str.split('batch'))==1):
            continue
        X = pd.read_csv(directory+'/'+batch_str,delimiter=',', engine='python')
        X['Pvalue'] = list(map(lambda x: 1 if x<=0.01 else 0,list(X['Pvalue'])))
        del X['CauseGene']
        del X['EffectGene']
        del X['Replicate']
        del X['Treatment']
        batch_arr.append(X)
    X_train = pd.concat(batch_arr)
    print(X_train.shape)
    return X_train

In [4]:
def making_x_test(directory):
    X_test = pd.read_csv(directory+'/test_data.csv',delimiter=',', engine='python')
    X_test['Pvalue'] = list(map(lambda x: 1 if x<=0.01 else 0,list(X_test['Pvalue'])))
    del X_test['CauseGene']
    del X_test['EffectGene']
    del X_test['Replicate']
    del X_test['Treatment']
    
    return X_test

In [2]:
main_directory = '/home/a20114261/sdelrio/GeneInteractions/GeneInteractionsBN_Datasets/Balanced_Batches/'

In [ ]:
main_directory = '/home/a20114261/sdelrio/GeneInteractions/GeneInteractionsBN_Datasets/Balanced_Batches/'
for directory in os.listdir(main_directory):
    if (directory == 'censored_Jnk'):
        continue
    print("***************************************************************************")
    print("Analyzing batches for "+directory+':')
    print("***************************************************************************")
    batches_dir = []
    print("- Appending bags routes")
    for batch_str in os.listdir(main_directory+directory):
        if (len(batch_str.split('batch'))==1):
            continue
        batches_dir.append(main_directory+directory+'/'+batch_str)
    results_arr = []
#     # making x_train
#     print("- Concatenating the training dataset for the AGA")
#     X_train = making_x_train(main_directory+directory)
#     # making x_test
#     print("- Reading the reserved test data sample for the AGA")
#     X_test = making_x_test(main_directory+directory)
    print("- Iterating over the bags:")
    for batch_str in batches_dir:
        print("***************************************************************************")
        print("\t- "+batch_str)
        print("***************************************************************************")
        results_arr.append(process_batch(batch_str))
    
    with open(main_directory+directory+'_results.pickle', 'wb') as f:
        # Pickle the 'data' dictionary using the highest protocol available.
        pickle.dump(results_arr, f, pickle.HIGHEST_PROTOCOL)
    

In [ ]:
for directory in os.listdir(main_directory):
    if ('.pickle' in directory):
        continue
    print(directory)
    if ('Jnk' in directory) or ('Ikk2' in directory):
        continue
    
    print("- Reading all batches for training data.")
    X_train = making_x_train(main_directory+directory)
    # making x_test
    print("- Reading the reserved test data sample for the AGA")
    
    X_test = making_x_test(main_directory+directory)
    
    with open(main_directory+directory+'_results.pickle', 'rb') as f:
        data = pickle.load(f)
    print(len(data))
    frec_array = np.zeros((541,)).tolist()
    for sub_data in data:
        sub_nodes = list(map(lambda x:int(x.split('@')[0].split('Node')[1])-1 ,sub_data['MMRWPC']['nodes']['Node1@2']['nodes']))
        for sub_node in sub_nodes:
            frec_array[sub_node] += 1
    print(frec_array)
    minimum_thresholds = [0,0.10,0.20,0.30]
    init_pobs = []
    for minimum_threshold in minimum_thresholds:
        init_pob = []
        for i in range(0,len(frec_array)):
            if (frec_array[i]>minimum_threshold*100):
                    init_pob.append(i)
        init_pobs.append({
            'init_pob':init_pob,
            'minimum':minimum_threshold
        })
    
    for init_pob in init_pobs:
            if (len(init_pob['init_pob'])==0):
                break
            ga_model = CrossAdaptiveGA(0.05,5,40,[basic_cross_sum,basic_cross_single_point,basic_cross_multi_point],[basic_bit_mutation])
            ga_model.fit(X_train, X_test, init_pob['init_pob'], 'Node1')
            set_result = ga_model.run(10)
            init_pob['results'] = set_result
            
    with open(main_directory+directory+'_aga_results.pickle', 'wb') as f:
        # Pickle the 'data' dictionary using the highest protocol available.
        pickle.dump(init_pobs, f, pickle.HIGHEST_PROTOCOL)
    

censored_Jnk
censored_Ikk2
censored_LEF1
- Reading all batches for training data.
(600000, 541)
- Reading the reserved test data sample for the AGA
100
[0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 1.0, 0.0, 1.0, 0.0, 6.0, 1.0, 2.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 7.0, 0.0, 0.0, 3.0, 1.0, 0.0, 4.0, 4.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 16.0, 0.0, 0.0, 1.0, 0.0, 14.0, 0.0, 2.0, 2.0, 1.0, 0.0, 0.0, 37.0, 0.0, 10.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 4.0, 1.0, 0.0, 0.0, 0.0, 0.0, 37.0, 9.0, 2.0, 0.0, 6.0, 7.0, 0.0, 0.0, 1.0, 0.0, 0.0, 8.0, 0.0, 0.0, 0.0, 1.0, 0.0, 2.0, 0.0, 5.0, 31.0, 0.0, 6.0, 0.0, 0.0, 4.0, 29.0, 1.0, 1.0, 8.0, 6.0, 0.0, 9.0, 0.0, 0.0, 0.0, 46.0, 1.0, 2.0, 0.0, 0.0, 0.0, 3.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 31.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 2.0, 0.0, 1.0, 2.0, 1.0, 0.0, 0.0, 0.0, 0.0, 8.0, 0.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 0.0, 7.0, 0.0, 1.0, 0.0, 0

Mutation!
Mutation!
Mutation!
Mutation!
Mutation!
Mutation!
Poblation change rate: 0.5648235031344121
Max score: 0.5779567865602052
Poblation change rate: 0.005547095787514866
Max score: 0.5779567865602052
Mutation!
Mutation!
Mutation!
Mutation!
Mutation!
Mutation!
Mutation!
Mutation!
Mutation!
Poblation change rate: 0.0037339308607199583
Max score: 0.5779567865602052
Mutation!
Mutation!
Mutation!
Mutation!
Mutation!
Mutation!
Mutation!
Mutation!
Mutation!
Mutation!
Mutation!
Mutation!
Mutation!
Mutation!
Mutation!
Poblation change rate: 0.003075318162147056
Max score: 0.5779567865602052
Poblation has been erased!
Mutation probability has been reseted!
Poblation change rate: -0.01027809427303925
Max score: 0.5779567865602052
Mutation!
Mutation!
Poblation change rate: 0.005050228198390805
Max score: 0.5795647295048119
Mutation!
Mutation!
Mutation!
Mutation!
Mutation!
Mutation!
Mutation!
Poblation change rate: 0.004781049254467984
Max score: 0.5795647295048119
Mutation!
Mutation!
Mutatio

Mutation!
Mutation!
Mutation!
Poblation change rate: 0.5937723803748296
Max score: 0.6037147477930856
Mutation!
Poblation change rate: 0.00489030201353069
Max score: 0.6037147477930856
Mutation!
Mutation!
Mutation!
Mutation!
Poblation change rate: 0.0027808198088203006
Max score: 0.6037147477930856
Mutation!
Mutation!
Mutation!
Mutation!
Mutation!
Mutation!
Mutation!
Mutation!
Poblation change rate: 0.0021343650488331533
Max score: 0.6037147477930856
Poblation has been erased!
Mutation probability has been reseted!
Mutation!
Mutation!
Poblation change rate: -0.0068673799594959565
Max score: 0.6037147477930856
Mutation!
Mutation!
Mutation!
Poblation change rate: 0.003730660881434922
Max score: 0.6070025893476886
Mutation!
Mutation!
Mutation!
Mutation!
Mutation!
Mutation!
Poblation change rate: 0.0030402706504220945
Max score: 0.6076488718218598
Mutation!
Mutation!
Mutation!
Mutation!
Mutation!
Mutation!
Mutation!
Mutation!
Mutation!
Mutation!
Mutation!
Mutation!
Mutation!
Poblation chan

In [9]:
# appending train and test data
train_dict = {}
test_dict = {}
for directory in os.listdir(main_directory):
    if ('.pickle' in directory):
        continue
    print(directory)
    train_dict[directory] = making_x_train(main_directory+directory)
    censored_data_name = directory.split('censored_')[1]
    X_test = pd.read_csv('../../GeneInteractionsBN_Datasets/DataGeneCausality/Tt/'+censored_data_name+'.csv',delimiter=';', engine='python')
    X_test['Pvalue'] = list(map(lambda x: 1 if x<=0.01 else 0,list(X_test['Pvalue'])))
    del X_test['CauseGene']
    del X_test['EffectGene']
    del X_test['Replicate']
    del X_test['Treatment']
    print(X_test.shape)
    test_dict[directory] = X_test
    

censored_Jnk
(600000, 541)
(300438, 541)
censored_Ikk2
(600000, 541)
(200292, 541)
censored_LEF1
(600000, 541)
(100146, 541)
censored_IRF4
(600000, 541)
(100146, 541)
censored_MYC
(600000, 541)
(100146, 541)
censored_Erk
(600000, 541)
(200292, 541)
censored_CTNNB1
(600000, 541)
(100146, 541)


In [10]:
for key in train_dict:
    train_dict[key].columns = ['Node'+str(x+1) for x in np.arange(0,541).tolist()]
for key in test_dict:
    test_dict[key].columns = ['Node'+str(x+1) for x in np.arange(0,541).tolist()]

In [15]:
with open(main_directory+'train_data.pickle', 'wb') as f:
        # Pickle the 'data' dictionary using the highest protocol available.
        pickle.dump(train_dict, f, pickle.HIGHEST_PROTOCOL)
with open(main_directory+'test_data.pickle', 'wb') as f:
        # Pickle the 'data' dictionary using the highest protocol available.
        pickle.dump(test_dict, f, pickle.HIGHEST_PROTOCOL)

In [3]:
with open(main_directory+'train_data.pickle', 'rb') as f:
    train_dict = pickle.load(f)
with open(main_directory+'test_data.pickle', 'rb') as f:
    test_dict = pickle.load(f)

In [4]:
def get_tops(key_list, f_selection_data):
    results_dict = {}
    
    for key in key_list:
        results = np.zeros((541,)).tolist()
        print(key)
        if('RW' not in key ):
            for sub_f in f_selection_data:
                for node in sub_f[key]['nodes']:
                    index = int(node[0].split('Node')[1])-1
                    value = node[1]                    
                    results[index] += value
            unsorted_list = list(zip(['Node'+str(x+1) for x in np.arange(0,541).tolist()],results))
            sorted_list = sorted(unsorted_list, key = lambda x : x[1], reverse =  True)            
        else:
            for sub_f in f_selection_data:
                for node in sub_f[key]['nodes']['Node1@2']['nodes']:                    
                    index = int(node.split('Node')[1].split('@')[0])-1
                    value = 1                    
                    results[index] += value
            unsorted_list = list(zip(['Node'+str(x+1) for x in np.arange(0,541).tolist()],results))
            sorted_list = sorted(unsorted_list, key = lambda x : x[1], reverse =  True)
        results_dict[key] = {
            'top2':sorted_list[:1],
            'top3':sorted_list[:3],
            'top5':sorted_list[:5],
            'top10':sorted_list[:10],
            'top15':sorted_list[:15]
        }
    return results_dict

In [5]:
def model_scorer(model_name,train_data, test_data, target_node, nodes_lists):
    
    results = {}
    for key in nodes_lists:
        X_train = train_data
        Y_train = train_data[target_node]
        if (model_name == 'NB'):
            clf = GaussianNB()
        elif (model_name == 'LR'):
            clf = LogisticRegression()
        elif (model_name == 'DTC'):
            clf = DecisionTreeClassifier()
        clf.fit(X_train[nodes_lists[key]],Y_train)

        precisions = []
        values_counter=set(train_data[target_node])
        for val in values_counter:
            if (len(test_data.query(target_node+'== '+str(val)))==0): 
                precisions.append(1)
                continue
            Y_pred_c = clf.predict(test_data.query(target_node+'== '+str(val))[nodes_lists[key]])
            precisions.append(Y_pred_c.tolist().count(val)/len(Y_pred_c))
        results[key] = reduce(operator.mul,precisions,1)**(1/len(precisions))
    return results

    

In [6]:
# taking the models for each case scenario
target_node = 'Node1'
model_name_arr=['NB','DTC','LR']
model_results_dict = {}

for directory in os.listdir(main_directory):
    if ('.pickle' in directory):
        continue
    print(directory)
    with open(main_directory+directory+'_results.pickle', 'rb') as f:
        f_selection_data = pickle.load(f)
    print(len(f_selection_data))
    
    key_list = list(f_selection_data[0].keys())
    results_dict = get_tops(key_list, f_selection_data)
    model_results_dict[directory] = {}
    print("======")
    for key in key_list:
        print(key)
        models_dict={}
        nodes_dict={}
        for t_key in results_dict[key]:
            nodes_dict[t_key] = [x[0] for x in results_dict[key][t_key]]
            
        for model_name in model_name_arr:
            models_dict[model_name] = model_scorer(model_name,train_dict[directory],test_dict[directory],target_node,nodes_dict)
            print(model_name)
            print(models_dict[model_name])
        model_results_dict[directory][key] = models_dict
        
    # aga_results
    with open(main_directory+directory+'_aga_results.pickle', 'rb') as f:
        aga_selection_data = pickle.load(f)
    model_results_dict[directory]['AGA'] = {}
    print("AGA")
    for i in range(0,len(aga_selection_data)):
        model_results_dict[directory]['AGA'][aga_selection_data[i]['minimum']] = {}
        nodes_dict = {}
        nodes_dict['unique'] = ['Node'+str(x[0]+1) for x in list(zip(np.arange(0,541).tolist(),aga_selection_data[i]['results']['set'])) if x[1]>0]
        #print(nodes_dict['unique'])
        print(aga_selection_data[i]['minimum'])
        for model_name in model_name_arr:
            print(model_name)
            model_results_dict[directory]['AGA'][aga_selection_data[i]['minimum']][model_name] = model_scorer(model_name,train_dict[directory],test_dict[directory],target_node,nodes_dict)
            print(model_results_dict[directory]['AGA'][aga_selection_data[i]['minimum']][model_name])

censored_Jnk
100
MI
TTEST_ind
TTEST_rel
MMRWPC
MI
NB
{'top2': 0.3728948729940539, 'top3': 0.4440618731665563, 'top5': 0.4615599811714931, 'top10': 0.5254033985320726, 'top15': 0.5199948960050162}
DTC
{'top2': 0.4969775804269178, 'top3': 0.4880699057864011, 'top5': 0.48869900123050747, 'top10': 0.3988290895355879, 'top15': 0.3774299063547912}
LR
{'top2': 0.4309699622922829, 'top3': 0.4974062653969121, 'top5': 0.5085865386145624, 'top10': 0.5323521405381548, 'top15': 0.5377886403048274}
TTEST_ind
NB
{'top2': 0.5096149266528566, 'top3': 0.5562624291270265, 'top5': 0.5494633472736403, 'top10': 0.546972322994281, 'top15': 0.5350700604132647}
DTC
{'top2': 0.5096149266528566, 'top3': 0.5562624291270265, 'top5': 0.5526321622027939, 'top10': 0.395847946693106, 'top15': 0.2989595571718091}
LR
{'top2': 0.5096149266528566, 'top3': 0.5493566301818759, 'top5': 0.5509321379397177, 'top10': 0.5467518436153866, 'top15': 0.5402648057015471}
TTEST_rel
NB
{'top2': 0.5096149266528566, 'top3': 0.55626242912

NB
{'top2': 0.3934433593414778, 'top3': 0.5230556288325152, 'top5': 0.55638654151298, 'top10': 0.5417838329788032, 'top15': 0.5411645277514557}
DTC
{'top2': 0.3934433593414778, 'top3': 0.42103767916034224, 'top5': 0.41055727419180843, 'top10': 0.4948571900495238, 'top15': 0.528896652674307}
LR
{'top2': 0.4454113682545874, 'top3': 0.48852760836603987, 'top5': 0.4744973767887608, 'top10': 0.4659515605394301, 'top15': 0.4423694385365605}
AGA
0
NB
{'unique': 0.48455294931600745}
DTC
{'unique': 0.31117196556202353}
LR
{'unique': 0.4906708145659787}
0.1
NB
{'unique': 0.5505419713556452}
DTC
{'unique': 0.45461085206461105}
LR
{'unique': 0.43425978284343586}
0.2
NB
{'unique': 0.5548423952524789}
DTC
{'unique': 0.46682313861245844}
LR
{'unique': 0.4509252103009812}
0.3
NB
{'unique': 0.45303744532943935}
DTC
{'unique': 0.36484475118084186}
LR
{'unique': 0.5032167676831476}
censored_MYC
100
MI
TTEST_ind
TTEST_rel
MMRWPC
MI
NB
{'top2': 0.2622475766994583, 'top3': 0.344731308392366, 'top5': 0.41079

In [ ]:
4+4

In [9]:
with open(main_directory+'total_results.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(model_results_dict, f, pickle.HIGHEST_PROTOCOL)